In [3]:
import csv, re, requests
from rdflib import Dataset, URIRef, Literal, Namespace, RDF, RDFS, OWL, XSD
from iribaker import to_iri
from SPARQLWrapper import SPARQLWrapper, JSON

In [4]:
TUTORIAL_REPOSITORY = "http://stardog.krw.d2s.labs.vu.nl/group12"

def upload_to_stardog(data):
    transaction_begin_url = TUTORIAL_REPOSITORY + "/transaction/begin"
    
    # Start the transaction, and get a transaction_id
    response = requests.post(transaction_begin_url, headers={'Accept': 'text/plain'})
    transaction_id = response.content
    
    # POST the data to the transaction
    post_url = TUTORIAL_REPOSITORY + "/" + transaction_id + "/add"
    response = requests.post(post_url, data=data, headers={'Accept': 'text/plain', 'Content-type': 'application/trig'})
    
    # Close the transaction
    transaction_close_url = TUTORIAL_REPOSITORY + "/transaction/commit/" + transaction_id
    response = requests.post(transaction_close_url)
    
    return str(response.status_code)

In [5]:
# A namespace for our resources
data = 'http://data.krw.d2s.labs.vu.nl/group12/resource/'
DATA = Namespace(data)
# A namespace for our vocabulary items (schema information, RDFS, OWL classes and properties etc.)
vocab = 'http://data.krw.d2s.labs.vu.nl/group12/vocab/'
VOCAB = Namespace('http://data.krw.d2s.labs.vu.nl/group12/vocab/')

# The URI for our graph
graph_uri = URIRef('http://data.krw.d2s.labs.vu.nl/group12/resource/trafficaccidentgraph')

# We initialize a dataset, and bind our namespaces
dataset = Dataset()
dataset.bind('g12data',DATA)
dataset.bind('g12vocab',VOCAB)

# We then get a new graph object with our URI from the dataset.
graph = dataset.graph(graph_uri)

# Create namespaces for our Geo-data (no geo-coordinates in these datasets)
# GSP = Namespace('http://www.opengis.net/ont/geosparql#')
# GSF = Namespace('http://www.opengis.net/ont/sf#')

In [6]:
VALID_CHARS = list("ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz0123456789-._~:/?#[]@!$&'()*+,;=")

# first load all (relevant) reference-files into memory
references = {}
FILENAMES = [
    #'aangrijppunten',
    'aardongevallen',
    'aflopen3',
    'aflopen4',
    'aflopen5',
    #'bewegingen',
    'bijzonderheden',
    'dagdelen',
    #'dagen',
    #'inrichtingen',
    #'leeftijdsklassen',
    'lichtgesteldheden',
    #'maanden',
    'manoeuvres',
    #'nationaliteiten',
    'toedrachten',
    'wegdekken',
    'wegsituaties',
    'wegverhardingen',
    'wegverlichtingen',
    'weersgesteldheden1',
    'weersgesteldheden2',
    'zichtafstanden',
    #'ziekenhuizen'
]

for fileName in FILENAMES:
    with open('../Milestone 1/data/' + fileName + '.csv', 'r') as refFile:
        temp = csv.reader(refFile, delimiter=',', quotechar='"')
        headers = temp.next()    # skip the headers
        references[fileName] = {}
        for row in temp:
            references[fileName][str(row[0])] = '::'.join(str(el) for el in row[1:])

In [7]:
# Load the externally defined schema into the default graph (context) of the dataset
dataset.default_context.parse('vocab.ttl', format='turtle')

# i = last two year digits (2009 - 2014)
for i in range(9, 14):
    if i < 10:
        i = '0' + str(i)
    
    count = 0
    
    myFile = open('../Milestone 1/data/ongevallen-20' + str(i) + '.csv', 'r')
    ongevallen = csv.reader(myFile, delimiter=',', quotechar='"')

    headers = ongevallen.next()
    headers[0] = headers[0].split('\xef\xbb\xbf')[1] if headers[0].startswith('\xef') else headers[0]
    
    ongevalIDX = headers.index('VKL_NUMMER')
    procesverbaalIDX = headers.index('PVOPGEM')
    datumIDX = headers.index('DATUM_VKL')
    tijdIDX = headers.index('TIJDSTIP')
    dagdeelIDX = headers.index('DDL_ID')
    afloop3IDX = headers.index('AP3_CODE')
    afloop4IDX = headers.index('AP4_CODE')
    afloop5IDX = headers.index('AP5_CODE')
    aantalslachtIDX = headers.index('ANTL_SLA')
    aantaldoodIDX = headers.index('ANTL_DOD')
    aantalgewondIDX = headers.index('ANTL_GZH')
    aantalspoedIDX = headers.index('ANTL_SEH')
    aantaloverigIDX = headers.index('ANTL_GOV')
    aantalpartijIDX = headers.index('ANTL_PTJ')
    aantaltoedrachtIDX = headers.index('ANTL_TDT')
    manoeuvreIDX = headers.index('MNE_CODE')
    aardIDX = headers.index('AOL_ID')
    wegsituatieIDX = headers.index('WSE_ID')
    bebkomIDX = headers.index('BEBKOM')
    maxsnelheidIDX = headers.index('MAXSNELHD')
    weglichtIDX = headers.index('WVL_ID')
    weghardIDX = headers.index('WVG_ID')
    wegdekIDX = headers.index('WDK_ID')
    lichtIDX = headers.index('LGD_ID')
    zichtIDX = headers.index('ZAD_ID')
    weer1IDX = headers.index('WGD_CODE_1')
    weer2IDX = headers.index('WGD_CODE_2')
    bijzonderVM1IDX = headers.index('BZD_ID_VM1')
    bijzonderVM2IDX = headers.index('BZD_ID_VM2')
    bijzonderVM3IDX = headers.index('BZD_ID_VM3')
    bijzonderIF1IDX = headers.index('BZD_ID_IF1')
    bijzonderIF2IDX = headers.index('BZD_ID_IF2')
    bijzonderIF3IDX = headers.index('BZD_ID_IF3')
    bijzonderTA1IDX = headers.index('BZD_ID_TA1')
    bijzonderTA2IDX = headers.index('BZD_ID_TA2')
    bijzonderTA3IDX = headers.index('BZD_ID_TA3')
    junctieIDX = headers.index('JTE_ID')
    wegvakIDX = headers.index('WVK_ID')
    huisnummerIDX = headers.index('HUISNUMMER')
    gemidIDX = headers.index('GME_ID')
    gemeenteIDX = headers.index('GME_NAAM')
    
    # NOTE: URI's with if-checks are optional and hence need to be checked (others are mandatory)
    
    for row in ongevallen:
        if row[gemidIDX] != '363': continue    # Amsterdam gem_id = 363
        
        # initialize optional fields
        dagdeel = ''
        wegsituatie = ''
        bebkom = ''
        maxsnelheid = ''
        weglicht = ''
        weghard = ''
        wegdek = ''
        licht = ''
        zicht = ''
        weer1 = ''
        weer2 = ''
        bijzonderVM1 = ''
        bijzonderVM2 = ''
        bijzonderVM3 = ''
        bijzonderIF1 = ''
        bijzonderIF2 = ''
        bijzonderIF3 = ''
        bijzonderTA1 = ''
        bijzonderTA2 = ''
        bijzonderTA3 = ''
        junctieid = ''
        wegvakid = ''
        huisnummer = ''
        
        # first create the URI's
        ongeval = URIRef(to_iri(data + 'TrafficAccident/' + row[ongevalIDX]))
        ongevalNaam = Literal('TrafficAccident' + row[ongevalIDX], datatype=XSD['string'])
        ongevalid = Literal(row[ongevalIDX], datatype=XSD['long'])
        procesverbaal = Literal(row[procesverbaalIDX] if row[procesverbaalIDX] != '' else 'N', lang='nl')
        if len(row[datumIDX]) == 8 and len(row[tijdIDX]) == 4:
            myDate = '-'.join([row[datumIDX][:4], row[datumIDX][4:6], row[datumIDX][6:]])
            myTime = ':'.join([row[tijdIDX][:2], row[tijdIDX][2:], '00'])
        elif len(row[datumIDX]) == 8:
            myDate = '-'.join([row[datumIDX][:4], row[datumIDX][4:6], row[datumIDX][6:]])
            myTime = '00:00:00'
        elif len(row[tijdIDX]) == 4:
            myDate = '20' + str(i) + '-01-01'
            myTime = ':'.join([row[tijdIDX][:2], row[tijdIDX][2:], '00'])
        else:
            myDate = '20' + str(i) + '-01-01'
            myTime = '00:00:00'
        myDateTime = Literal(myDate + 'T' + myTime, datatype=XSD['dateTime'])
        if row[dagdeelIDX] != '':
            dagdeel = Literal(references['dagdelen'][row[dagdeelIDX]], datatype=XSD['string'])
        afloop3 = Literal(references['aflopen3'][row[afloop3IDX]], lang='nl')
        afloop4 = Literal(references['aflopen4'][row[afloop4IDX]], lang='nl')
        afloop5 = Literal(references['aflopen5'][row[afloop5IDX]], lang='nl')
        aantalslacht = Literal(row[aantalslachtIDX], datatype=XSD['int'])
        aantaldood = Literal(row[aantaldoodIDX], datatype=XSD['int'])
        aantalgewond = Literal(row[aantalgewondIDX], datatype=XSD['int'])
        aantalspoed = Literal(row[aantalspoedIDX], datatype=XSD['int'])
        aantaloverig = Literal(row[aantaloverigIDX], datatype=XSD['int'])
        aantalpartij = Literal(row[aantalpartijIDX], datatype=XSD['int'])
        aantaltoedracht = Literal(row[aantaltoedrachtIDX], datatype=XSD['int'])
        manoeuvre = Literal(references['manoeuvres'][row[manoeuvreIDX]], lang='nl')
        aard = Literal(references['aardongevallen'][row[aardIDX]], lang='nl')
        if row[wegsituatieIDX] != '':
            wegsituatie = Literal(references['wegsituaties'][row[wegsituatieIDX]], lang='nl')
        if row[bebkomIDX] != '':
            bebkom = Literal(row[bebkomIDX], lang='nl')
        if row[maxsnelheidIDX] != '':
            maxsnelheid = Literal(row[maxsnelheidIDX], datatype=XSD['int'])
        if row[weglichtIDX] != '':
            weglicht = Literal(references['wegverlichtingen'][row[weglichtIDX]], lang='nl')
        if row[weghardIDX] != '':
            weghard = Literal(references['wegverhardingen'][row[weghardIDX]], lang='nl')
        if row[wegdekIDX] != '':
            wegdek = Literal(references['wegdekken'][row[wegdekIDX]], lang='nl')
        if row[lichtIDX] != '':
            licht = Literal(references['lichtgesteldheden'][row[lichtIDX]], lang='nl')
        if row[zichtIDX] != '':
            zicht = Literal(references['zichtafstanden'][row[zichtIDX]], lang='nl')
        if row[weer1IDX] != '':
            weer1 = Literal(references['weersgesteldheden1'][row[weer1IDX]], lang='nl')
        if row[weer2IDX] != '':
            weer2 = Literal(references['weersgesteldheden2'][row[weer2IDX]], lang='nl')
        if row[bijzonderVM1IDX] != '':
            bijzonderVM1 = Literal(references['bijzonderheden'][row[bijzonderVM1IDX]], lang='nl')
        if row[bijzonderVM2IDX] != '':
            bijzonderVM2 = Literal(references['bijzonderheden'][row[bijzonderVM2IDX]], lang='nl')
        if row[bijzonderVM3IDX] != '':
            bijzonderVM3 = Literal(references['bijzonderheden'][row[bijzonderVM3IDX]], lang='nl')
        if row[bijzonderIF1IDX] != '':
            bijzonderIF1 = Literal(references['bijzonderheden'][row[bijzonderIF1IDX]], lang='nl')
        if row[bijzonderIF2IDX] != '':
            bijzonderIF2 = Literal(references['bijzonderheden'][row[bijzonderIF2IDX]], lang='nl')
        if row[bijzonderIF3IDX] != '':
            bijzonderIF3 = Literal(references['bijzonderheden'][row[bijzonderIF3IDX]], lang='nl')
        if row[bijzonderTA1IDX] != '':
            bijzonderTA1 = Literal(references['bijzonderheden'][row[bijzonderTA1IDX]], lang='nl')
        if row[bijzonderTA2IDX] != '':
            bijzonderTA2 = Literal(references['bijzonderheden'][row[bijzonderTA2IDX]], lang='nl')
        if row[bijzonderTA3IDX] != '':
            bijzonderTA3 = Literal(references['bijzonderheden'][row[bijzonderTA3IDX]], lang='nl')
        if row[junctieIDX] != '':
            junctie = URIRef(to_iri(data + 'Junction/' + row[junctieIDX]))
            junctieNaam = Literal('Junction' + row[junctieIDX], datatype=XSD['string'])
            junctieid = Literal(row[junctieIDX], datatype=XSD['int'])
        if row[wegvakIDX] != '':
            wegvak = URIRef(to_iri(data + 'RoadSection/' + row[wegvakIDX]))
            wegvakNaam = Literal('RoadSection' + row[wegvakIDX], datatype=XSD['string'])
            wegvakid = Literal(row[wegvakIDX], datatype=XSD['int'])
        if row[huisnummerIDX] != '':
            huisnummer = Literal(row[huisnummerIDX], datatype=XSD['int'])
        gemid = Literal(row[gemidIDX], datatype=XSD['int'])
        gemeente = Literal(row[gemeenteIDX], lang='nl')
        gemeenteNaam = ''.join([c for c in row[gemeenteIDX].replace(' ', '_') if c in VALID_CHARS])
        dorp = URIRef(to_iri(data + gemeenteNaam))
        
        # second create the actual triples
        graph.add((ongeval, RDF.type, VOCAB['TrafficAccident']))
        graph.add((ongeval, RDFS.label, ongevalNaam))
        graph.add((ongeval, VOCAB['vkl_nummer'], ongevalid))
        graph.add((ongeval, VOCAB['pvopgem'], procesverbaal))
        graph.add((ongeval, VOCAB['from'], myDateTime))
        graph.add((ongeval, VOCAB['to'], myDateTime))
        if dagdeel != '':
            graph.add((ongeval, VOCAB['ddl_id'], dagdeel))
        graph.add((ongeval, VOCAB['ap3_code'], afloop3))
        graph.add((ongeval, VOCAB['ap4_code'], afloop4))
        graph.add((ongeval, VOCAB['ap5_code'], afloop5))
        graph.add((ongeval, VOCAB['antl_sla'], aantalslacht))
        graph.add((ongeval, VOCAB['antl_dod'], aantaldood))
        graph.add((ongeval, VOCAB['antl_gzh'], aantalgewond))
        graph.add((ongeval, VOCAB['antl_seh'], aantalspoed))
        graph.add((ongeval, VOCAB['antl_gov'], aantaloverig))
        graph.add((ongeval, VOCAB['antl_ptj'], aantalpartij))
        graph.add((ongeval, VOCAB['antl_tdt'], aantaltoedracht))
        graph.add((ongeval, VOCAB['mne_code'], manoeuvre))
        graph.add((ongeval, VOCAB['aol_id'], aard))
        if wegsituatie != '':
            graph.add((ongeval, VOCAB['wse_id'], wegsituatie))
        if bebkom != '':
            graph.add((ongeval, VOCAB['bebkom'], bebkom))
        if maxsnelheid != '':
            graph.add((ongeval, VOCAB['maxsnelhd'], maxsnelheid))
        if weglicht != '':
            graph.add((ongeval, VOCAB['wvl_id'], weglicht))
        if weghard != '':
            graph.add((ongeval, VOCAB['wvg_id'], weghard))
        if wegdek != '':
            graph.add((ongeval, VOCAB['wdk_id'], wegdek))
        if licht != '':
            graph.add((ongeval, VOCAB['lgd_id'], licht))
        if zicht != '':
            graph.add((ongeval, VOCAB['zad_id'], zicht))
        if weer1 != '':
            graph.add((ongeval, VOCAB['wgd_code_1'], weer1))
        if weer2 != '':
            graph.add((ongeval, VOCAB['wgd_code_2'], weer2))
        if bijzonderVM1 != '':
            graph.add((ongeval, VOCAB['bzd_id_vm1'], bijzonderVM1))
        if bijzonderVM2 != '':
            graph.add((ongeval, VOCAB['bzd_id_vm2'], bijzonderVM2))
        if bijzonderVM3 != '':
            graph.add((ongeval, VOCAB['bzd_id_vm3'], bijzonderVM3))
        if bijzonderIF1 != '':
            graph.add((ongeval, VOCAB['bzd_id_if1'], bijzonderIF1))
        if bijzonderIF2 != '':
            graph.add((ongeval, VOCAB['bzd_id_if2'], bijzonderIF2))
        if bijzonderIF3 != '':
            graph.add((ongeval, VOCAB['bzd_id_if3'], bijzonderIF3))
        if bijzonderTA1 != '':
            graph.add((ongeval, VOCAB['bzd_id_ta1'], bijzonderTA1))
        if bijzonderTA2 != '':
            graph.add((ongeval, VOCAB['bzd_id_ta2'], bijzonderTA2))
        if bijzonderTA3 != '':
            graph.add((ongeval, VOCAB['bzd_id_ta3'], bijzonderTA3))
        if huisnummer != '':
            graph.add((ongeval, VOCAB['huisnummer'], huisnummer))
        if wegvakid != '':
            graph.add((wegvak, RDF.type, VOCAB['RoadSection']))
            graph.add((wegvak, RDFS.label, wegvakNaam))
            graph.add((wegvak, VOCAB['wvk_id'], wegvakid))
            graph.add((ongeval, VOCAB['roadSection'], wegvak))
        if junctieid != '':
            graph.add((junctie, RDF.type, VOCAB['Junction']))
            graph.add((junctie, RDFS.label, junctieNaam))
            graph.add((junctie, VOCAB['jte_id'], junctieid))
            graph.add((ongeval, VOCAB['junction'], junctie))
        graph.add((dorp, RDF.type, VOCAB['Town']))
        graph.add((dorp, VOCAB['gme_naam'], gemeente))
        graph.add((dorp, VOCAB['gme_id'], gemid))
        graph.add((ongeval, VOCAB['town'], dorp))
    
        count = count + 1
        if count % 400 == 0:
            print 'response.status_code = ' + upload_to_stardog(dataset.serialize(format='trig'))
            
            # We reset the dataset, and bind our namespaces
            dataset = Dataset()
            dataset.bind('g12data',DATA)
            dataset.bind('g12vocab',VOCAB)

            # We then get a new graph object with our URI from the dataset.
            graph = dataset.graph(graph_uri)
            
    # close the csv file
    myFile.close()

# upload the (final) data
print 'response.status_code = ' + upload_to_stardog(dataset.serialize(format='trig'))

response.status_code = 200
response.status_code = 200
response.status_code = 200
response.status_code = 200
response.status_code = 200
response.status_code = 200
response.status_code = 200
response.status_code = 200
response.status_code = 200
response.status_code = 200
response.status_code = 200
response.status_code = 200
response.status_code = 200
response.status_code = 200
response.status_code = 200
response.status_code = 200
response.status_code = 200
response.status_code = 200
response.status_code = 200
response.status_code = 200
response.status_code = 200
response.status_code = 200
response.status_code = 200
response.status_code = 200
response.status_code = 200
response.status_code = 200
response.status_code = 200
response.status_code = 200
response.status_code = 200
response.status_code = 200
response.status_code = 200
response.status_code = 200
response.status_code = 200
response.status_code = 200
response.status_code = 200
response.status_code = 200
response.status_code = 200
r

In [53]:
#with open('./test-rdf.trig','w') as f:
#    dataset.serialize(f, format='trig')